# Data LLM

In [2]:
# %run ../utils/db_loader.py ../data/processed/inventory.csv ../db/database.db INVENTORY_TABLE

In [3]:
import sqlite3
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain import hub
from langgraph.prebuilt import create_react_agent

In [4]:
# Connect to database
conn = sqlite3.connect('../db/database.db')

# Load table into DataFrame
df = pd.read_sql_query("SELECT * FROM INVENTORY_TABLE LIMIT 10", conn)
display(df)

# Close connection
conn.close()


,id,Material Code,Material Description,Unit,Date,Open Stock,Material Issued,Material Received,Closing Stock,Prefix,Type,PDT,BFP,Price
0,1,400015,BAG; CANVAS TOOL 16'' LENGTH,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,90,0,750.0
1,2,400088,MACHINE; DRILL ELECTRIC WITH 13 BITS,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,90,0,1.0
2,3,400167,GOGGLE; FOR GAS WELDING,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,80,1,181.29
3,4,400176,"HAMMER; BALL PEEN, W/HANDLE 1LBS",EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,120,0,1126.86
4,5,400177,"HAMMER; BALL PEEN, W/HANDLE 1.5LBS",EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,120,0,1833.0
5,6,400178,"HAMMER; BALL PEEN, ENGG,W/HANDLE, 2LBS",EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,90,0,1085.0
6,7,400180,HAMMER; MASON CHIESEL W/HANDLE 2-1/2LBS,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,150,0,6572.0
7,8,400205,"PLIER; INSULATED 8""",EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,60,1,1.0
8,9,400207,PLIER; NOSE COMBINATION 8'',EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,150,0,7905.0
9,10,400220,TAPE; STEEL FLEXIBLE 10' 3M,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,0,1,280.0


In [40]:
# Load environment variables from .env file
load_dotenv('../.env')

# Get the OpenAI API key from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
langsmith_api_key = os.getenv('LANGCHAIN_API_KEY')

if openai_api_key is None:
    raise ValueError("OpenAI API key not found in environment variables")

In [6]:
# Connect to your SQLite database
db = SQLDatabase.from_uri("sqlite:///../db/database.db")

In [7]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [8]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [38]:
# Read the prompt from a text file
with open('./conf/prompt.txt', 'r') as file:
    prompt_text = file.read()

In [28]:
# Retrieve the prompt template
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
system_message = prompt_template.format(dialect="SQLite", top_k=5) + ' ' + prompt_text

# Create the agent executor
agent_executor = create_react_agent(
    llm,
    toolkit.get_tools(),
    state_modifier=system_message
)

In [36]:
# Define your natural language query
natural_language_query = "What is the total value of the Closing Stock on 1st January 2024?"

# Execute the query
events = agent_executor.stream(
    {"messages": [("user", natural_language_query)]},
    stream_mode="values",
)

# Capture the final AI message
final_message = None
for event in events:
    final_message = event["messages"][-1]
    #event["messages"][-1].pretty_print()

# Extract and print the result
if final_message:
    print(final_message.content) 

The total value of the Closing Stock on 1st January 2024 is approximately PKR 2,679,595,841.04.
